# Importing the libraries

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

# Setting Image Dimensions

In [4]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

# Loading and Preprocessing the Dataset

Path of the training images data

In [ ]:
# Load the dataset 

dataset_dir = r""

In [9]:



filenames = os.listdir(dataset_dir)

categories = []
for f_name in filenames:
    category = f_name.split('.')[0]
    # Assuming your defective images are labeled as 'defective' and non-defective as 'non_defective'
    if category == 'defective':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df

# Creating the Convolutional Neural Network (CNN) Model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(Image_Width, Image_Height, Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
model.summary()

# Setting Callbacks for Model Training

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]


# Preparing the Dataset

In [14]:
# 'non-defective' and 'defective' respectively
df["category"] = df["category"].replace({0: 'non-defective', 1: 'defective'})

# Split the DataFrame into training and validation sets
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)

# Reset the index of the training and validation DataFrames
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

# Get the total number of samples in the training and validation sets
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

# Define the batch size for training
batch_size = 15


# Data Augmentation and Image Data Generators

### Provide the train data images path and test data images path

In [ ]:
train_data = r""

test_data = r""

In [ ]:

# Get the list of filenames in the test directory
test_filenames = os.listdir(test_data)

# Create a DataFrame for the test data
test_df = pd.DataFrame({
    'filename': test_filenames
})


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    
    
    # train data
    train_data,  
    
    
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

# Validation data generator (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    
    
    # train data 
    train_data,  
    
    
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

# Test data generator (similar to validation, no augmentation, only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    
#   test data 
    test_data,  
    
    
    x_col='filename',
    target_size=Image_Size,
    batch_size=batch_size,
    class_mode=None,  # Set class_mode to None since the DataFrame does not contain labels
    shuffle=False  # Set shuffle to False to ensure that predictions are in the same order as filenames
)


# Compiling and Training the Model

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Recreate the optimizer
optimizer = RMSprop()

# Compile the model with the new optimizer
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Resume training
epochs = 1
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)


# Saving the Trained Model

In [ ]:
model.save("model1_industrial_equipment_labelled_10epoch.h5")

# Visualizing Sample Images from the Dataset

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img

sample_test = df.head(18)
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    # Assuming 'category' is the column containing class labels
    category = row['category']
    
    # Correcting the file path with single forward slashes
    img_path = r"" + filename
    
    
    img = load_img(img_path, target_size=Image_Size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()


# Predicting Class Label for a Single Image

## Testing the model :: 

### Tasting 1 :

### Upload the image path there


In [ ]:
image = r""          

In [ ]:
from PIL import Image
import numpy as np

# Load and preprocess the image                    

im = Image.open(image)
im = im.resize(Image_Size)
im = np.expand_dims(im, axis=0)
im = np.array(im)
im = im / 255

# Predict the class probabilities
pred_probs = model.predict(im)[0]

# Extract the class with the highest probability
pred_class = np.argmax(pred_probs)


results = {0: 'non-defective', 1: 'defective'}  
pred_label = results[pred_class]

print("Predicted class:", pred_class)
print("Predicted label:", pred_label)


# GUI for Defective Equipment Classification

### Tasting 2 :

### Load the saved model path : 

In [ ]:
modelpath = r""

In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import ImageTk, Image
import numpy as np
from keras.models import load_model

# Load the pre-trained model


model = load_model(modelpath) 

# Define the class labels
classes = {0: 'non-defective', 1: 'defective'}

# Initialize GUI
top = tk.Tk()
top.geometry('800x600')
top.title('Defective Equipment Classification')
top.configure(background='#CDCDCD')

# Define the label for displaying the prediction
label = Label(top, background='#CDCDCD', font=('arial', 15, 'bold'))
sign_image = Label(top)

# Function to classify the uploaded image
def classify(file_path):
    try:
        # Load and preprocess the image
        image = Image.open(file_path)
        image = image.resize((128, 128))
        image = np.expand_dims(image, axis=0)
        image = np.array(image)
        image = image / 255

        # Predict the class probabilities
        pred_probs = model.predict(image)[0]

        # Extract the class with the highest probability
        pred_class = np.argmax(pred_probs)

        # Map the class index to its corresponding label
        pred_label = classes[pred_class]

        # Update the label with the predicted class
        label.configure(foreground='#011638', text=pred_label)
    except Exception as e:
        print("An error occurred during classification:", e)
        label.configure(foreground='#011638', text="An error occurred during classification.")

# Function to display the "Classify Image" button
def show_classify_button(file_path):
    classify_b = Button(top, text="Classify Image", command=lambda: classify(file_path), padx=10, pady=5)
    classify_b.configure(background='#364156', foreground='white', font=('arial', 10, 'bold'))
    classify_b.place(relx=0.79, rely=0.46)

# Function to upload an image
def upload_image():
    try:
        file_path = filedialog.askopenfilename()
        uploaded = Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width() / 2.25), (top.winfo_height() / 2.25)))
        im = ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image = im
        label.configure(text='')
        show_classify_button(file_path)
    except Exception as e:
        print("An error occurred during image upload:", e)

# Button to upload an image
upload = Button(top, text="Upload an image", command=upload_image, padx=10, pady=5)
upload.configure(background='#364156', foreground='white', font=('arial', 10, 'bold'))
upload.pack(side=tk.BOTTOM, pady=50)

# Label to display the uploaded image
sign_image.pack(side=tk.BOTTOM, expand=True)

# Label to display the classification result
label.pack(side=tk.BOTTOM, expand=True)

# Heading label
heading = Label(top, text="Defective Equipment Classification", pady=20, font=('arial', 20, 'bold'))
heading.configure(background='#CDCDCD', foreground='#364156')
heading.pack()

# Run the Tkinter event loop
top.mainloop()